In [76]:
import great_expectations as gx
import pandas as pd
import warnings
import datetime as datetime
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
df = pd.read_csv("./data/transactions.csv")

# Regex for amount(Any number + "." + Any number): ^\d+\.\d+$
amount_pattern = r"^-?\d+\.\d+$"
# Regex for currency(Three uppercase letters): ^[A-Z]{3}$
currency_pattern = r"^[A-Z]{3}$"
# Regex for timestamp format: ^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$
timestamp_pattern = r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"

# Fixes wrong spacing in amount
def keep_only_last_dot(s):
    if pd.isna(s):
        return s
    s = str(s).replace(" ", "").replace(".", "")  # Remove spaces and existing dots
    if len(s) > 2:
        return s[:-2] + "." + s[-2:]
    return s

# Apply to DataFrame
df["amount"] = df["amount"].apply(keep_only_last_dot)

# Fixes amount
df["amount"] = pd.to_numeric(df["amount"])  # Convert to numeric
df["amount"] = df["amount"].astype(float) # Convert to float

# Fixes currency (removes spaces)
df["currency"] = df["currency"].astype(str).str.replace(" ", "")

formats_to_try_currency = [
    "SEK",
    "DKK",
    "USD",
    "EUR",
    "NOK",
    "RMB",
    "ZAR",
    "GBP",
    "ZMW",
    "JPY"
]

# Fixes datetime format
formats_to_try_datetime = [
    "%Y%m%d %H:%M:%S",
    "%y-%m-%d %H:%M:%S",
    "%Y-%m-%d %H:%M",
    "%Y-%m-%d %H.%M",
    "%Y-%m-%d %H.%M:%S",
    "%Y-%m-%d %H.%M.%S",
    "%Y-%m-%d %H:%M:%S",
    "%Y.%m.%d %H.%M.%S"
]

def parse_and_format(date_str):
    for fmt in formats_to_try_datetime:
        try:
            return datetime.datetime.strptime(date_str, fmt).strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            continue
    return None

df["timestamp"] = df["timestamp"].apply(parse_and_format)

# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="transactions_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="transactions_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations
validator.expect_column_values_to_be_between("amount", min_value=0.01, max_value=100000)
validator.expect_column_values_to_not_be_null("amount")
validator.expect_column_values_to_match_regex("amount", regex=amount_pattern)

validator.expect_column_values_to_match_regex("currency", regex=currency_pattern)
validator.expect_column_values_to_be_in_set("currency", value_set=formats_to_try_currency)

validator.expect_column_values_to_match_regex("timestamp", regex=timestamp_pattern)
validator.expect_column_values_to_not_be_null("timestamp")

# Validate
results = validator.validate(result_format="COMPLETE")

# Print results
print(results)

# Checks results for any unexpected counts.
# Put Unexpected into an invalid DF and expected into a valid DF
# (Currently only invalid transactions are those not using correct regex format or over a certain amount)

unexpected_transactions = [
    invalid_index
    for result in results["results"]
    for invalid_index in result["result"].get("unexpected_index_list", [])
]

invalid_transactions = df.iloc[unexpected_transactions] # List to iloc
valid_transactions = df.drop(index=unexpected_transactions) # Drop invalid rows from valid df

print(f"Invalid Transactions: {len(invalid_transactions)}")
print(f"Valid Transactions: {len(valid_transactions)}")

df = df.drop(index=unexpected_transactions)

print(f"Dropped {len(invalid_transactions)} Invalid Transactions from df.")
print(f"Current Transactions in df: {len(df)}")


C:\Users\Nevai\AppData\Local\Temp\ipykernel_340\2791070369.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/transactions.csv")
Calculating Metrics: 100%|██████████| 45/45 [00:00<00:00, 52.53it/s] 

{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "pandas-transactions_data",
          "column": "amount",
          "min_value": 0.01,
          "max_value": 100000.0
        },
        "meta": {}
      },
      "result": {
        "element_count": 100000,
        "unexpected_count": 1,
        "unexpected_percent": 0.001,
        "partial_unexpected_list": [
          3700879.0
        ],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.001,
        "unexpected_percent_nonmissing": 0.001,
        "partial_unexpected_counts": [
          {
            "value": 3700879.0,
            "count": 1
          }
        ],
        "partial_unexpected_index_list": [
          1614
        ],
        "unexpected_list": [
          3700879.0
        ],
        "unexpected_index_list": [
          1614
   

In [77]:
exchange_rate_to_sek = {
    'SEK': 1.0,
    'DKK': 1.46,
    'USD': 9.68,
    'EUR': 10.89,
    'NOK': 0.94,
    'RMB': 1.34,
    'ZAR': 0.54,
    'GBP': 12.94,
    'ZMW': 0.36,
    'JPY': 0.067
}

exchange_rate_from_sek = {
    cur: 1/rate for cur, rate in exchange_rate_to_sek.items()
}

df['exchange_rate_to_sek'] = df['currency'].map(exchange_rate_to_sek)

df['amount_in_sek'] = df['amount'] * df['exchange_rate_to_sek']

df['exchange_rate_from_sek'] = df['currency'].map(exchange_rate_from_sek)

df['amount_from_sek'] = df['amount'] * df['exchange_rate_from_sek']

df[['amount', 'currency', 'amount_in_sek', 'amount_from_sek']]

,amount,currency,amount_in_sek,amount_from_sek
0,7746.03,SEK,7746.0300,7746.030000
1,45193.04,SEK,45193.0400,45193.040000
2,33029.71,SEK,33029.7100,33029.710000
3,35994.61,SEK,35994.6100,35994.610000
4,32229.73,SEK,32229.7300,32229.730000
...,...,...,...,...
99995,146.04,USD,1413.6672,15.086777
99996,56.98,USD,551.5664,5.886364
99997,128.34,USD,1242.3312,13.258264
99998,149.13,USD,1443.5784,15.405992
